In [0]:
# Standard numpy for matrices, vectors, etc..
import numpy as np

# Visualisation (plotting, etc..)
import matplotlib.pyplot as plt
%matplotlib inline

# New for today! Tensorflow from Google: 
# https://www.tensorflow.org
import tensorflow as tf

# Please see the following page for getting 
# started guide and tutorials:
# https://www.tensorflow.org/get_started/

try:
    in_colab = False
    import google.colab
    in_colab = True
except:
    pass

# Use the following to access tensorboard when running on colab
if in_colab:
    !pip install -U tensorboardcolab
    from tensorboardcolab import *

In [0]:
# What we are used to in standard programming:

a = np.array([1.0, 2.0, 3.0])

b = np.array([2.0, 2.0, 2.0])

a_plus_b = a + b

a_power_b = a ** b

c = a_plus_b * a_power_b

print('In numpy:')
print('a = ', a)
print('a = ', b)
print('a + b = ', a_plus_b)
print('a ** b = ', a_power_b)
print('c = (a + b) * (a ** b) = ', c)

In [0]:
# Let's do this with tensorflow:
#
# (Using t_ to denote tensorflow)
#

# Reset tensorflow!
tf.reset_default_graph()

t_a = tf.constant(a, name='a')

t_b = tf.constant(b, name='b')

t_a_plus_b = t_a + t_b

t_a_power_b = t_a ** t_b

t_c = t_a_plus_b * t_a_power_b

print('In tensorflow:')
print('a = ', t_a)
print('a = ', t_b)
print('a + b = ', t_a_plus_b)
print('a ** b = ', t_a_power_b)
print('c = (a + b) * (a ** b) = ', t_c)

In [0]:
# Can ignore this code for now, this just creates a 
# visualisation file so we can see what is going on!

with tf.Session() as session:
    if in_colab:
        tbc = TensorBoardColab()
        summary_file_writer = tbc.get_writer()
        summary_file_writer.add_graph(session.graph)
        summary_file_writer.flush()
        tbc.close()
    else:
        summary_file_writer = tf.summary.FileWriter(
            'visualisation_files', session.graph)
        summary_file_writer.flush()

# This has created a folder called "visualisation_files" 
# with some data in it that we can view with the command:
#
# tensorboard --logdir=visualisation_files
#
# If in colab, the TensorBoardColab call will output a link to follow
#

### This has created a computational graph!

You can see the graph in tensorboard from the previous cell.

### Why would we want to do this?

...

In [0]:
# First - does it actually work?

# In order to use the graph we have to "run" it!
# In tensorflow, we need to run things inside of a session

# This line creates a session..
with tf.Session() as session:
    # Inside here we can use the "session" object created..
    
    # Let's run our graph to actually compute something!
    result = session.run(t_c)
    
    print("result = ", result)
    
# Outside of the "with" statement the session object is 
# deleted and we can no longer use it
#
# This line would cause an error:
# result_again = session.run(t_c)

In [0]:
# Let's check everything makes sense:

print('In numpy:')
print('a = ', a)
print('a = ', b)
print('a + b = ', a_plus_b)
print('a ** b = ', a_power_b)
print('c = (a + b) * (a ** b) = ', c)

with tf.Session() as session:
    print('In tensorflow session:')   
    print('a = ', session.run(t_a))
    print('a = ', session.run(t_b))
    print('a + b = ', session.run(t_a_plus_b))
    print('a ** b = ', session.run(t_a_power_b))
    print('c = (a + b) * (a ** b) = ', session.run(t_c))

# Everything should be the same!

### So it seems to work!

### But isn't this more effort than the numpy version?

...

### What if we are doing an optimistion?

We want to fit a Gaussian distribution $\mathcal{N}(\mu, \sigma^2)$ to a set of numbers $X = \{ x_0, x_1, \dots, x_{N-1} \}$

If we assume the numbers are i.i.d. (indentically and independently distributed) samples from a Gaussian then the likelihood of $X$ is given by:

\begin{align}
p(X) &= p(x_0) \cdot p(x_1) \cdot \dots \cdot p(x_{N-1}) \\
 &= \mathcal{N}(x_0 \,|\, \mu, \sigma^2) \cdot \mathcal{N}(x_1 \,|\, \mu, \sigma^2) \cdot \dots \cdot \mathcal{N}(x_{N-1} \,|\, \mu, \sigma^2) \\
 &= \prod_{n=0}^{N-1} \mathcal{N}(x_{n} \,|\, \mu, \sigma^2) \\
 &= \prod_{n=0}^{N-1} \frac{1}{\sqrt{2\pi\sigma^2}} 
    \exp{\left( - \frac{(x_{n} - \mu)^2}{2\sigma^2} \right)}
\end{align}

**Top Tip!** When working with exponential family of distributions it often helps to work in the log domain..

\begin{align}
\log  p(X)  &= \sum_{n=0}^{N-1} 
    -\frac{1}{2} \log{\left( 2\pi\sigma^2 \right)}
    -\frac{(x_{n} - \mu)^2}{2\sigma^2}
\end{align}

So, we have the *maximum likelihood* fit to the parameters when we find the values of $\mu$ and $\sigma^2$ that maximise $p(X)$ which (since $\log\,(\cdot)$ is a concave function) occurs at the same time that $\log p(X)$ is maximised.

In our case we can find an analytic solution for

\begin{align}
\mu^{*} &= {\arg\max}_{\mu} \, \log p(X) \\
{\sigma^{*}}^2 &= {\arg\max}_{\sigma^2} \, \log p(X) \\
\end{align}

But let's pretend that the problem was more complicated and we needed to use *optimisation* to solve the problem..

To perform numerical optimisation we need to be able to calculate gradients of the objective function ($\log p(X)$) wrt the parameters that you are optimising ($\mu$ and $\sigma^2$).

*Let's see how to do this in tensorflow..*

In [0]:
# First let's generate some numbers to fit the data to..

# How many values of x?
N = 20

# Pick the real mean and variance..
mu_true = 2.5
sigma_true = 1.5

x_n = np.random.normal(mu_true, sigma_true, N)

np.set_printoptions(precision=3, linewidth=50)
print('X = \n', np.transpose(x_n))

We are now going to build our tensorflow graph but we are going to account for the fact that $\mu$ and $\sigma^2$ are no longer constants since we wish to vary their values to find the maximum of $\log p(X)$. With numerical optimisation, we need to start with a guess for the values of $\mu$ and $\sigma^2$; in this case, we will start with
\begin{align}
\mu_{\mathrm{initial}} &= 1\\
\sigma^2_{\mathrm{initial}} &= 1
\end{align}

**Top Tip!** Care needs to be taken with $\sigma$ since it can only be a positive value (unlike $\mu$ which can be any real number). In general, tensorflow variables can be positive or negative. In this example we square the value of `t_sigma` before using it to ensure that `t_sigma_2` is a positive value but we shouldn't, therefore, use the value for `t_sigma` directly in calculations..

As a reminder, we want to find:
\begin{align}
\log  p(X)  &= \sum_{n=0}^{N-1} 
    -\frac{1}{2} \log{\left( 2\pi\sigma^2 \right)}
    -\frac{(x_{n} - \mu)^2}{2\sigma^2} 
%    \\
%    &=  -\frac{N}{2} \log{\left( 2\pi\sigma^2 \right)}
%    - \frac{1}{2\sigma^2} \sum_{n=0}^{N-1}\left(x_{n} - \mu\right)^2
\end{align}

In [0]:
# Reset tensorflow to remove our old a, b, etc..
tf.reset_default_graph()

# Our initial guesses..
mu_initial_guess = 1.0
sigma_initial_guess = np.sqrt(1.0)

# The data to fit to
t_x_n = tf.constant(x_n, name='X')

# Note: mu and sigma are now *variables* not constants!
# We need to specify their data type and initial value.. 
t_mu = tf.Variable(mu_initial_guess, 
                   dtype=tf.float64, 
                   name="mu")
t_sigma = tf.Variable(sigma_initial_guess, 
                      dtype=tf.float64, 
                      name="sigma")

# Note: this step is important - don't use t_sigma directly!! 
t_sigma_2 = t_sigma ** 2.0

# Calculate log p(X) terms..

t_x_minus_mu_2 = (t_x_n - t_mu) ** 2.0
t_denom = 2.0 * t_sigma_2
t_sigma_term = - 0.5 * tf.log(2.0 * np.pi * t_sigma_2)

t_log_P_terms = t_sigma_term - (t_x_minus_mu_2 / t_denom)

# The sum is performed by a reduction in tensorflow 
# (since a vector goes in and a scalar comes out)
# but this is effectively the same as np.sum(...)
t_log_P = tf.reduce_sum(t_log_P_terms)

In [0]:
# Let's just check that we calculated things correctly:

with tf.Session() as session:
    # IMPORTANT! Need to run this at the start to
    # initialise the values for the variables
    # t_mu and t_sigma. You will get an error if 
    # you forget!
    session.run(tf.global_variables_initializer())
    
    test_value = session.run(t_log_P)
    print('Tensorflow log p(X) = ', test_value)
    print('(using initial guesses for mu and sigma)\n')

# Check with scipy..
from scipy.stats import norm
check_value = np.sum(norm.logpdf(x_n, 
                                 mu_initial_guess, 
                                 sigma_initial_guess))
print('Value from scipy stats package = ', check_value)

assert(np.isclose(test_value, check_value))

print('\nEverything working!')

In [0]:
# We can even go crazy and check with different 
# values of the parameters..

mu_new_test_value = 3.3
sigma_new_test_value = 0.5

with tf.Session() as session:
    # IMPORTANT! (see above..)
    session.run(tf.global_variables_initializer())
    
    # Change the values of the variables while the
    # session is running..
    session.run(t_mu.assign(mu_new_test_value))
    session.run(t_sigma.assign(sigma_new_test_value))
    
    test_value = session.run(t_log_P)
    print('New tensorflow log p(X) = ', test_value)
    print('(using new values for mu and sigma)\n')

# Check with scipy..
from scipy.stats import norm
check_value = np.sum(norm.logpdf(x_n, 
                                 mu_new_test_value, 
                                 sigma_new_test_value))
print('New value from scipy stats package = ', check_value)

assert(np.isclose(test_value, check_value))

print('\nEverything working!')

*So finally, we get to the advantage of tensorflow!*

Now, we can calculate the objective function and we can calculate the value of the objective when changing the input parameters. 

This is great for optimisation (since we are going to need to change the parameters to increase the objective) but what we really need for the optimisation is to calculate the **gradient of the objective wrt to the parameters**. Let's see how to do that in tensorflow..

In [0]:
with tf.Session() as session:
    # IMPORTANT! (see above..)
    session.run(tf.global_variables_initializer())
    
    t_gradient_wrt_mu = tf.gradients(t_log_P, 
                                     t_mu)
    t_gradient_wrt_sigma = tf.gradients(t_log_P, 
                                        t_sigma)
    
    grad_mu = session.run(t_gradient_wrt_mu)
    grad_sigma = session.run(t_gradient_wrt_sigma)
    
    print('Gradient wrt mu = ', grad_mu)
    print('Gradient wrt sigma = ', grad_sigma)

Shall we check that result. Remember we have:

\begin{align}
\log  p(X)  &= \sum_{n=0}^{N-1} 
    -\frac{1}{2} \log{\left( 2\pi\sigma^2 \right)}
    -\frac{(x_{n} - \mu)^2}{2\sigma^2} \\
    &=  -\frac{N}{2} \log{\left( 2\pi\sigma^2 \right)}
    - \frac{1}{2\sigma^2} \sum_{n=0}^{N-1}\left(x_{n} - \mu\right)^2
\end{align}

So for $\mu$ we have:

\begin{align}
\frac{\partial \log  p(X)}{\partial \mu}   
    &= - 0
    - \frac{1}{2\sigma^2} \frac{\partial}{\partial \mu}  \sum_{n=0}^{N-1} \left(x_{n} - \mu\right)^2 \\
    &= - \frac{1}{2\sigma^2} \sum_{n=0}^{N-1} \frac{\partial}{\partial \mu} \left(x_{n} - \mu\right)^2 \\
    &= - \frac{1}{2\sigma^2} \sum_{n=0}^{N-1} 2 \left(x_{n} - \mu\right) \frac{\partial}{\partial \mu}\left(x_{n} - \mu\right)  \\
    &= \frac{1}{\sigma^2} \sum_{n=0}^{N-1} \left(x_{n} - \mu\right)
\end{align}

where we used the "chain rule" a number of times..

In [0]:
# numpy check of gradient wrt mu

grad_mu_check = np.sum(x_n - mu_initial_guess) / \
                (sigma_initial_guess ** 2)
    
print('Our analytic gradient wrt mu = ', grad_mu_check)

print('Tensorflow gradient wrt mu = ', grad_mu)

assert(np.isclose(grad_mu, grad_mu_check))

print('\nExcellent! tensorflow calculated the gradient for us :)')

## Everyone should now be in awe!

This might seem like something trivial but hopefully you can see that actually quite a lot of maths and then coding went into determining the gradient. 

In fact, you can do the same to check the value for the gradient wrt $\sigma^2$.

When we calculated the result using the chain rule. Since tensorflow built up a graph of the operations, it is able to apply the chain rule results for us automatically.

This:
\begin{align}
\log  p(X)  &= \sum_{n=0}^{N-1} 
    -\frac{1}{2} \log{\left( 2\pi\sigma^2 \right)}
    -\frac{(x_{n} - \mu)^2}{2\sigma^2} 
%    \\
%    &=  -\frac{N}{2} \log{\left( 2\pi\sigma^2 \right)}
%    - \frac{1}{2\sigma^2} \sum_{n=0}^{N-1}\left(x_{n} - \mu\right)^2
\end{align}
has become the computational graph that you can see in tensorboard below.

For example, the `pow` operation represents $r = a^b$ for the inputs $a,b$ and result $r$. Tensorflow then knows that $\frac{\partial r}{\partial a} = b a^{b-1}$, and by chaining these operations together it can work backwards through the graph (from $\log p(X)$ at the top to $\mu$ at the bottom) to calculate the gradient.

Therefore, the tensorflow graph has multiple uses. A forward pass can calculate the objective for the current set of parameters and a backwards pass can calculate the gradients of an objective wrt any of the parameters.

In [0]:
# Look at the computational graph in tensorboard..
if in_colab:
    tbc = TensorBoardColab()
    summary_file_writer = tbc.get_writer()
    summary_file_writer.add_graph(session.graph)
    summary_file_writer.flush()
    tbc.close()
else:
    summary_file_writer = tf.summary.FileWriter(
        'visualisation_files', session.graph)
    summary_file_writer.flush()

## But the fun doesn't end here!

In fact tensorflow has actually done all the work to do the optimisation part, not just calculate the derivatives. So we can now run a full optimisation with our graph and it will use the gradients internally.

In [0]:
# Create a gradient descent optimiser that uses a
# certain step size (learning_rate)..
optimiser = tf.train.GradientDescentOptimizer(learning_rate=0.05)

# We want to maximise log p(X) therefore we 
# need to minimise - log p(X)
t_objective = - t_log_P

# We want to optimise wrt mu and sigma
vars_to_optimise = [t_mu, t_sigma]

minimize_operation = optimiser.minimize(t_objective,
                                       var_list=vars_to_optimise)

# Number of iterations to perform
num_iterations = 50

with tf.Session() as session:
    # IMPORTANT! (see above..)
    session.run(tf.global_variables_initializer())
    
    # Run a number of iterations of gradient descent..
    for iteration in range(num_iterations):
        # At each iteration evaluate the minimize_operation
        # to perform the gradient descent step and also
        # keep track of the current value..
        step, cost = session.run([minimize_operation, t_log_P])
        
        # Print out the value of log P every 10 iterations..
        if ((iteration + 1) % 10 == 0):
            print('iter %4d, log P(X) = %0.3f' % 
                  (iteration + 1, cost))
    
    # Get the final results of the optimisation..
    mu_optimised = session.run(t_mu)
    sigma_optimised = session.run(t_sigma)
    
    print('\nAfter optimisation:')
    print('Tensorflow mu = ', mu_optimised)
    print('Tensorflow sigma = ', sigma_optimised)

print('\nAnalytic estimates:')
print('Estimated mu = ', np.mean(x_n))
print('Estimated std = ', np.std(x_n))

print('\nGround truth values:')
print('True mu = ', mu_true)
print('True sigma = ', sigma_true)

## Excellent! So we agree with the analytic estimate :)

Of course, the values don't match the true estimate since we didn't have a very large sample size.

What if we want to run again with more samples?

Unfortunately, we made `t_x_n` a constant at the start of our tensorflow code so now we can't change it. Instead, we could have made it a `placeholder`. This tells tensorflow "there will be some data here but I'm going to give it to you later".

*How do we give the data later on?*

We can provide the values to placeholders by specifying a "feed dictionary" to `session.run`. This means, "during this session use the following values to replace all the placeholders".

Let's do our example again..

In [0]:
# Reset tensorflow to remove our old a, b, etc..
tf.reset_default_graph()

# THIS TIME USE A PLACEHOLDER!
#
# The data to fit to is provided as a placeholder.
# We need to tell it what type of data we will provide..
t_x_n = tf.placeholder(dtype=tf.float64, name='X')

# EVERYTHING ELSE IS AS IT WAS BEFORE..

# Note: mu and sigma are now *variables* not constants!
# We need to specify their data type and initial value.. 
t_mu = tf.Variable(mu_initial_guess, 
                   dtype=tf.float64, 
                   name="mu")
t_sigma = tf.Variable(sigma_initial_guess, 
                      dtype=tf.float64, 
                      name="sigma")

# Note: this step is important - don't use t_sigma directly!! 
t_sigma_2 = t_sigma ** 2.0

# Calculate log p(X) terms..

t_x_minus_mu_2 = (t_x_n - t_mu) ** 2.0
t_denom = 2.0 * t_sigma_2
t_sigma_term = - 0.5 * tf.log(2.0 * np.pi * t_sigma_2)

t_log_P_terms = t_sigma_term - (t_x_minus_mu_2 / t_denom)

# The sum is performed by a reduction in tensorflow 
# (since a vector goes in and a scalar comes out)
# but this is effectively the same as np.sum(...)
t_log_P = tf.reduce_sum(t_log_P_terms)

In [0]:
# NOW WHEN WE RUN WE NEED TO FILL IN THE PLACEHOLDER..

# Create a gradient descent optimiser that uses a
# certain step size (learning_rate)..
optimiser = tf.train.GradientDescentOptimizer(learning_rate=0.05)

# We want to maximise log p(X) therefore we 
# need to minimise - log p(X)
t_objective = - t_log_P

# We want to optimise wrt mu and sigma
vars_to_optimise = [t_mu, t_sigma]

minimize_operation = optimiser.minimize(t_objective,
                                       var_list=vars_to_optimise)

# Number of iterations to perform
num_iterations = 50

with tf.Session() as session:
    # IMPORTANT! (see above..)
    session.run(tf.global_variables_initializer())
    
    # Run a number of iterations of gradient descent..
    for iteration in range(num_iterations):
        # At each iteration evaluate the minimize_operation
        # to perform the gradient descent step and also
        # keep track of the current value..
        #
        # NEED TO ADD THE FEED DICTIONARY OTHERWISE WE
        # DON'T KNOW WHAT VALUE TO USE FOR t_x_n..
        #
        step, cost = session.run([minimize_operation, t_log_P],
                                 feed_dict={ t_x_n : x_n })
        
        # Print out the value of log P every 10 iterations..
        if ((iteration + 1) % 10 == 0):
            print('iter %4d, log P(X) = %0.3f' % 
                  (iteration + 1, cost))
    
    # Get the final results of the optimisation..
    mu_optimised = session.run(t_mu)
    sigma_optimised = session.run(t_sigma)
    
    print('\nAfter optimisation:')
    print('Tensorflow mu = ', mu_optimised)
    print('Tensorflow sigma = ', sigma_optimised)

print('\nAnalytic estimates:')
print('Estimated mu = ', np.mean(x_n))
print('Estimated std = ', np.std(x_n))

print('\nGround truth values:')
print('True mu = ', mu_true)
print('True sigma = ', sigma_true)

We can now even make this a function and call it with lots of different data..

In [0]:
def find_parameters_using_tensorflow(x_input, 
                                     learning_rate=0.05):
    # Create a gradient descent optimiser that uses a
    # certain step size (learning_rate)..
    optimiser = tf.train.GradientDescentOptimizer(
        learning_rate=learning_rate)

    # We want to maximise log p(X) therefore we 
    # need to minimise - log p(X)
    t_objective = - t_log_P

    # We want to optimise wrt mu and sigma
    vars_to_optimise = [t_mu, t_sigma]

    minimize_operation = optimiser.minimize(t_objective,
                                           var_list=vars_to_optimise)

    # Number of iterations to perform
    num_iterations = 50

    with tf.Session() as session:
        # IMPORTANT! (see above..)
        session.run(tf.global_variables_initializer())

        # Run a number of iterations of gradient descent..
        for iteration in range(num_iterations):
            # At each iteration evaluate the minimize_operation
            # to perform the gradient descent step and also
            # keep track of the current value..
            #
            # PASS THE ARGUMENT TO THE FUNCTION INTO THE FEED
            # DICTIONARY..
            #
            step, cost = session.run([minimize_operation, t_log_P],
                                     feed_dict={ t_x_n : x_input })

            # Print out the value of log P every 10 iterations..
            if ((iteration + 1) % 10 == 0):
                print('iter %4d, log P(X) = %0.3f' % 
                      (iteration + 1, cost))

        # Get the final results of the optimisation..
        mu_optimised = session.run(t_mu)
        sigma_optimised = session.run(t_sigma)

    return mu_optimised, sigma_optimised

In [0]:
# Let's try with a larger N

N_bigger = 1000

x_bigger = np.random.normal(mu_true, sigma_true, N_bigger)

new_mu, new_sigma = find_parameters_using_tensorflow(x_bigger, 
                                                     learning_rate=0.001)

print('Tensorflow estimates:')
print('Tensorflow mu = ', new_mu)
print('Tensorflow sigma = ', new_sigma)

print('\nAnalytic estimates:')
print('Estimated mu = ', np.mean(x_bigger))
print('Estimated std = ', np.std(x_bigger))

print('\nGround truth values:')
print('True mu = ', mu_true)
print('True sigma = ', sigma_true)

## All sorts of more advanced topics

- Visualise parts of computation (e.g. Tensorboard)
- Reusable components (e.g. modules for neural networks / classifiers / etc..)
- Run computations on the GPU instead of the CPU (often faster)
- Easy to scale; can distribute computations over an entire cluster!